In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jupyros
import rospy
import math
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time

In [ ]:
rospy.init_node("superturtle")

# Turtlesim

In [ ]:
# !rosrun turtlesim turtlesim_node
# !rosrun turtlesim turtle_teleop_key

In [ ]:
turtle = jupyros.TurtleWidget()
display(turtle.canvas)

### Subscriber

In [ ]:
topic_name = '/turtle1/pose'

# Based on the dimensions of window controlled by teleop keyboard
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * turtle.canvas_width
    y_convert = (convert_factor - y) / convert_factor * turtle.canvas_height
    
    return x_convert, y_convert

def callback_move_turtle(msg):
    elapsed_time = time() - turtle.last_move_time

    if elapsed_time > 0.05:
        x, y = convert_xy_units(msg.x, msg.y)
        position = {"x": x, "y": y, "theta": msg.theta}
        turtle.move_to_pose(position)

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtle(msg))

### Publisher

In [ ]:
%%thread_cell

rate = rospy.Rate(5)
pub = rospy.Publisher(topic_name, Pose, queue_size=10, latch=True)
for i in range(360*5):
    msg = Pose()
    msg.x = (i/1800*5.504)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5.504)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.publish(msg)
    rate.sleep()